In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
model_tag = 'llama3'  
dataset_tag = '20_newsgroups'
prompt = 'emb'
temp = 0
number_of_experimet = 1
selected_samples =  'gaussian'

In [6]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [66]:
dataset_for_annotation.columns

Index(['label', 'text'], dtype='object')

In [67]:
print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

CohereLLM
Params: {}         label                                               text
0    Business  Coke is it: BHP coal to double BHP Billiton pl...
1    Business  BIG HIKE IN ENERGY BILLS The company says the ...
2    Business  Officials Blame Contractors in Tunnel Leak The...
3    Business  Before the Bell: Krispy Kreme Down 11 Pct Shar...
4      Sports  Medical Examiner Finds No Injuries on Thanou-S...
..        ...                                                ...
995     World  Indian politician sent to prison Top Hindu nat...
996    Sports  Ravens - Bills: Game Notes COMPOSED: With Lewi...
997  Sci/Tech  Dell touts in-home services Dell offers to ins...
998    Sports  Lately, it's been fast times World Series anal...
999     World  Venezuela Raises Taxes on Orinoco Deals &lt;p&...

[1000 rows x 2 columns] Task: Read the following text and determine the most appropriate category based on its content and the examples below.

Categories:
- World
- Sports
- Business
- Sci/Tech

Ex

In [68]:
text_col = "text"
label_col = "label"
output_col = "output"

In [69]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation,
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [70]:
df = annotator.get_results(is_save=True, output_path=output_path)

Nr: 0 Predicted label: Business
Nr: 1 Predicted label: Business
Nr: 2 Predicted label: Business
Nr: 3 Predicted label: Business
Nr: 4 Predicted label: Sports
Nr: 5 Predicted label: Business
Nr: 6 Predicted label: Business
Nr: 7 Predicted label: Business
Nr: 8 Predicted label: Sci/Tech
Nr: 9 Predicted label: World
Nr: 10 Predicted label: Business
Nr: 11 Predicted label: World
Nr: 12 Predicted label: Business
Nr: 13 Predicted label: Sci/Tech
Nr: 14 Predicted label: World
Nr: 15 Predicted label: Sci/Tech
Nr: 16 Predicted label: Sports
Nr: 17 Predicted label: Sci/Tech
Nr: 18 Predicted label: Sci/Tech
Nr: 19 Predicted label: Sports
Nr: 20 Predicted label: Business
Nr: 21 Predicted label: World
Nr: 22 Predicted label: Sports
Nr: 23 Predicted label: Business
Nr: 24 Predicted label: World
Nr: 25 Predicted label: World
Nr: 26 Predicted label: World
Nr: 27 Predicted label: Sci/Tech
Nr: 28 Predicted label: Sci/Tech
Nr: 29 Predicted label: Sci/Tech
Nr: 30 Predicted label: Sports
Nr: 31 Predicted l

In [71]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/emb_temp0/cohere/ag_news_cohere_emb_gaussian_temp0_exp1.csv


In [7]:
df = pd.read_csv(output_path)
df

,Unnamed: 0,text,output,logprobs,top_logprobs,original_label
0,0,article ashish arora writes excerpts netnewssc...,talk.politics.misc,"{'content': [{'token': 'talk', 'logprob': -0.0...","[{'token': 'talk', 'logprob': -0.0001187254019...",talk.politics.misc
1,1,gateway telepath modem month actually one woul...,comp.os.ms-windows.misc,"{'content': [{'token': 'comp', 'logprob': -0.1...","[{'token': 'comp', 'logprob': -0.1082649305462...",comp.sys.ibm.pc.hardware
2,2,anybody provide advice concerning following tw...,sci.med,"{'content': [{'token': 'sci', 'logprob': -6.89...","[{'token': 'sci', 'logprob': -6.89005973981693...",sci.med
3,3,article mike silverman writes anybody know goi...,rec.sport.baseball,"{'content': [{'token': 'rec', 'logprob': -0.00...","[{'token': 'rec', 'logprob': -0.00032574593205...",rec.sport.baseball
4,4,article stich christian e writes installed mot...,sci.electronics,"{'content': [{'token': 'sci', 'logprob': -0.00...","[{'token': 'sci', 'logprob': -0.00110219744965...",sci.electronics
...,...,...,...,...,...,...
1981,1981,david sternlight writes article karl barrus wr...,sci.crypt,"{'content': [{'token': 'sci', 'logprob': -0.17...","[{'token': 'sci', 'logprob': -0.17696242034435...",sci.crypt
1982,1982,hello im looking information alphanumeric page...,sci.electronics,"{'content': [{'token': 'sci', 'logprob': -0.02...","[{'token': 'sci', 'logprob': -0.02991333045065...",sci.electronics
1983,1983,john r daker writes would like offocially nomi...,rec.autos,"{'content': [{'token': 'rec', 'logprob': -0.08...","[{'token': 'rec', 'logprob': -0.08121661096811...",rec.motorcycles
1984,1984,article writes looking information concerning ...,sci.space,"{'content': [{'token': 'sci', 'logprob': -0.00...","[{'token': 'sci', 'logprob': -0.00035386974923...",sci.space


In [8]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_first_bracketed(text):
        match = re.search(r'\[([^\[\]]+?)\]', str(text))
        return match.group(1) if match else None
    
    df[write_col] = df[read_col].apply(extract_first_bracketed)
    return df


In [9]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    # df = extract_bracketed_word(df, output_col, 'extracted_label')
    df.to_csv(path, index=False)
    
    return df


In [10]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,Unnamed: 0,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples
0,0,article ashish arora writes excerpts netnewssc...,talk.politics.misc,"{'content': [{'token': 'talk', 'logprob': -0.0...","[{'token': 'talk', 'logprob': -0.0001187254019...",talk.politics.misc,True
1,1,gateway telepath modem month actually one woul...,comp.os.ms-windows.misc,"{'content': [{'token': 'comp', 'logprob': -0.1...","[{'token': 'comp', 'logprob': -0.1082649305462...",comp.sys.ibm.pc.hardware,False
2,2,anybody provide advice concerning following tw...,sci.med,"{'content': [{'token': 'sci', 'logprob': -6.89...","[{'token': 'sci', 'logprob': -6.89005973981693...",sci.med,True
3,3,article mike silverman writes anybody know goi...,rec.sport.baseball,"{'content': [{'token': 'rec', 'logprob': -0.00...","[{'token': 'rec', 'logprob': -0.00032574593205...",rec.sport.baseball,True
4,4,article stich christian e writes installed mot...,sci.electronics,"{'content': [{'token': 'sci', 'logprob': -0.00...","[{'token': 'sci', 'logprob': -0.00110219744965...",sci.electronics,True
...,...,...,...,...,...,...,...
1981,1981,david sternlight writes article karl barrus wr...,sci.crypt,"{'content': [{'token': 'sci', 'logprob': -0.17...","[{'token': 'sci', 'logprob': -0.17696242034435...",sci.crypt,False
1982,1982,hello im looking information alphanumeric page...,sci.electronics,"{'content': [{'token': 'sci', 'logprob': -0.02...","[{'token': 'sci', 'logprob': -0.02991333045065...",sci.electronics,False
1983,1983,john r daker writes would like offocially nomi...,rec.autos,"{'content': [{'token': 'rec', 'logprob': -0.08...","[{'token': 'rec', 'logprob': -0.08121661096811...",rec.motorcycles,False
1984,1984,article writes looking information concerning ...,sci.space,"{'content': [{'token': 'sci', 'logprob': -0.00...","[{'token': 'sci', 'logprob': -0.00035386974923...",sci.space,False
